In [2]:
!pip install mne

Importing the Necessary Libraries

In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim  as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
import matplotlib.pyplot as plt
import mne

In [1]:
# Load the dataset and defining the directory
data_dir = '/content/drive/MyDrive/Dataset_new'

In [5]:
# Data loading and preprocessing
def load_and_preprocess_edf(file_path, duration=10, overlap=5, num_channels=32, target_length=2500):
    raw = mne.io.read_raw_edf(file_path, preload=True) # read the raw files and preload them
    raw.filter(l_freq=0.5, h_freq=50) # defining the base and notch frequency
    data = raw.get_data()
    data = mne.filter.resample(data, down=raw.info['sfreq'] / 250)

    if data.shape[0] < num_channels:
        data = np.pad(data, ((0, num_channels - data.shape[0]), (0, 0)), mode='constant')
    elif data.shape[0] > num_channels:
        data = data[:num_channels, :]

    epochs = []
    for start in range(0, data.shape[1] - duration * 250, overlap * 250):
        end = start + duration * 250
        epoch = data[:, start:end]

        if epoch.shape[1] < target_length:
            epoch = np.pad(epoch, ((0, 0), (0, target_length - epoch.shape[1])), mode='constant')
        elif epoch.shape[1] > target_length:
            epoch = epoch[:, :target_length]

        epochs.append(epoch)

    return np.array(epochs)

In [2]:
def load_dataset(data_dir):
    X, y = [], []
    for label, state in enumerate(['rest', 'task']): # loads the data from the directories task and rest state
        state_dir = os.path.join(data_dir, state)
        for file in os.listdir(state_dir):
            if file.endswith('.edf'):
                file_path = os.path.join(state_dir, file)
                epochs = load_and_preprocess_edf(file_path)
                X.extend(epochs)
                y.extend([label] * len(epochs)) # tells the code to return 0 for rest and task for 1

    return np.array(X), np.array(y)

### helps manage the EEG dataset and convert to pytorch tensors

In [7]:

class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

### Starting the Vision transformer task of creating patch embedings for transformers to read the eeg dataset

In [8]:
# Model definition
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, patch_size, embed_dim):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        return x.flatten(2).transpose(1, 2)

### Start the main vision transformer class where we define all the layers and along with a forward method

In [9]:

class VisionTransformer(nn.Module):
    def __init__(self, in_channels, patch_size, embed_dim, num_heads, num_layers, num_classes):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.patch_embed = PatchEmbedding(in_channels, patch_size, embed_dim)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_dim, num_heads, dim_feedforward=embed_dim * 4,batch_first=True),
            num_layers
        )
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        num_patches = x.shape[1] - 1
        pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, self.embed_dim)).to(x.device)
        x = x + pos_embed

        x = self.transformer(x)
        x = x[:, 0]
        x = self.fc(x)
        return x

Defining an epoch function with the optimiser the loss and loss calculation along each epoch

In [10]:
# Training and evaluation functions
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        if inputs.dim() == 3:
            inputs = inputs.unsqueeze(1)
        B, C, H, W = inputs.shape
        if H != 32 or W != 2500:
            inputs = F.interpolate(inputs, size=(32, 2500), mode='bilinear', align_corners=False)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

Evaluation function wehere the dropout layers are disabled and gradient is also not calculated

In [11]:
def evaluate(model, dataloader, device):
    model.eval()
    all_preds = [] # predicted tables
    all_labels = [] # true tables
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            if inputs.dim() == 3:
                inputs = inputs.unsqueeze(1)
            B, C, H, W = inputs.shape
            if H != 32 or W != 2500:
                inputs = F.interpolate(inputs, size=(32, 2500), mode='bilinear', align_corners=False)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_preds, all_labels

Loading the dataset parsing all the directories and loading all the EEG dataset

In [12]:
# Main execution
if __name__ == "__main__":
    # Load and prepare data
    data_dir = '/content/drive/MyDrive/Dataset_new'
    X, y = load_dataset(data_dir)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    train_dataset = EEGDataset(X_train, y_train)
    test_dataset = EEGDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/rest/Subject11_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 s)

Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/rest/Subject00_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 s)

Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/rest/Subject01_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 s)

Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/rest/Subject10_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 93999  =      0.000 ...   187.998 secs...


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/task/Subject07_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 s)

Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/task/Subject06_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/task/Subject04_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 s)

Extracting EDF parameters from /content/drive/MyDrive/Dataset_new/task/Subject09_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Initialising if there is cuda/gpu and defining the learning rate and epochs

In [13]:
    # Initialize model and training parameters
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VisionTransformer(in_channels=1, patch_size=5, embed_dim=128, num_heads=8, num_layers=6, num_classes=2).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 10

# Was Unable to run the final training cell because the cell was crashing all the time

- Tried changing the batch size and everything else I could but could not bring myself to train the model
- Maybe need better gpu

In [ ]:
    # Training loop
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

    # Evaluation
    test_preds, test_labels = evaluate(model, test_loader, device)

    # Calculate metrics
    accuracy = accuracy_score(test_labels, test_preds)
    precision = precision_score(test_labels, test_preds, average='binary')
    recall = recall_score(test_labels, test_preds, average='binary')
    f1 = f1_score(test_labels, test_preds, average='binary')
    cm = confusion_matrix(test_labels, test_preds)

    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-score: {f1:.4f}')

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    # Print classification report
    from sklearn.metrics import classification_report
    print("\nClassification Report:")
    print(classification_report(test_labels, test_preds, target_names=['Rest', 'Task']))

    # Plot ROC curve
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(test_labels, test_preds)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()